In [1]:
import pandas as pd
import torchaudio
import torch
import numpy as np 
import random
import os, sys

/home/srini/.conda/envs/speechbrain/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Folder to find .wav files
# data_folder="/mnt/datadrive/datasets/spkveri/mrt1/spk450/wav/"
data_folder="/mnt/results/silverbullet/audios_500"
audio_dir = "/mnt/results/silverbullet/audios_500"

data_dir = "/mnt/results/silverbullet/data/mr10"
#Spk2utt mapping path
utt2spk = data_dir + "/utt2spk"
#wav.scp path
wavscp = data_dir + "/wav.scp"

#Path to save the preprocessed data
output_folder ="/mnt/results/silverbullet/exp/mr10_v1/save/"

#Percentage split for train, dev, test respectively 
split=[0.6,0.2, 0.2]

In [3]:
#-->For the above cell, Write a python code to check if output_folder dir exists, if not create new dir or throw error

In [4]:
def split_fn (df: pd.DataFrame, split):
    
    split_indices = [int(len(df) * percentage_split) for percentage_split in split]
    
    #Select rows
    print(split_indices)
    print("next index:", split_indices[1]+split_indices[0])
    
    train_df = df.iloc[:split_indices[0],:]
    validation_df = df.iloc[split_indices[0]:(split_indices[0]+split_indices[1])]
    test_df = df.iloc[(split_indices[0]+split_indices[1]):,:]
    enrol_df = df.iloc[(split_indices[0]+split_indices[1]):,:]

    print("Length of train_df", len(train_df))
    print("Length of dev",len(validation_df))
    print("Length of test", len(test_df))
    
    #print(train_df.head())
    #print(validation_df.head())
    
    return train_df, validation_df, test_df, enrol_df

In [5]:
def veri_test (test_df: pd.DataFrame, output_folder):

    final_df=pd.DataFrame()

    #matched samples

    matched_df = pd.DataFrame()
    n_matched_samples = 27

    def extract_id(tmp_df):
        _id = test_df[test_df["wav"] == tmp_df]['ID'].values[0]
        return _id

    def extract_spk_id(tmp_df):
        spkr_id = test_df[test_df["wav"] == tmp_df]['spk_id'].values[0]
        return spkr_id
    
    for i in range(n_matched_samples):

        groups= test_df.groupby("spk_id")

        def select_random_pairs(group):
            wav_files = group['wav'].tolist()
            random_pairs = random.sample(wav_files, 2)
            return random_pairs

        # Apply the function to each group and convert the result to a list
        random_pairs_list = groups.apply(select_random_pairs).tolist()

        #print("Iteration")
        #print(len(random_pairs_list))
        #print(random_pairs_list)

        shufmatched_df = pd.DataFrame(random_pairs_list, columns = ['wav', 'shuffled_column'])
        #print(len(shuf_df))
        #print(shuf_df.head(2))

        matched_df = matched_df.append(shufmatched_df, ignore_index=True) 

    print("no of matched pairs",len(matched_df))
#     print(test_df.head())
    print(matched_df.head(2))

    
    
    
    # Apply the function to the 'wav' column and store the results in two new columns
#     matched_df['ID'], matched_df['spk_id'] = matched_df['wav'].apply(extract_id).str
    matched_df["ID"] = matched_df["wav"].apply(extract_id)
    matched_df["spk_ID"] = matched_df["wav"].apply(extract_spk_id)
    
    matched_df["ID_shuf"] = matched_df["shuffled_column"].apply(extract_id)
    matched_df["spk_ID_shuf"] = matched_df["shuffled_column"].apply(extract_spk_id)
    
    new_columns = ['ID', 'spk_id','ID_shuf','spk_id_shuf', 'wav',"shuffled_column"]
    matched_df = matched_df.reindex(columns=new_columns)

    print(matched_df.head())


    #Unmatched sample
    unmatched_df = pd.DataFrame()
    n_unmatch_samples = 10
    for i in range(n_unmatch_samples):

        #sample one audio from each spk_id randomly 

        df_random = test_df.groupby('spk_id').apply(lambda x: x.sample(1)).drop_duplicates().reset_index(drop=True)
        #print(df_random)

        wav_list = df_random['wav'].tolist()
        f_list=[]

        for i in range(1104):
            l= random.sample(wav_list,2)
            #print(l)
            f_list.append(l)
        #print(f_list)
    
        shuf_df = pd.DataFrame(f_list, columns = ['wav', 'shuffled_column'])
        #print(shuf_df)

        # Apply the function to the 'wav' column and store the results in two new columns
#         shuf_df['ID'], shuf_df['spk_id'] = shuf_df['wav'].apply(extract_id).str

        shuf_df['ID'] = shuf_df["wav"].apply(extract_id)
        shuf_df['spk_id'] = shuf_df["wav"].apply(extract_spk_id)
        shuf_df['ID_shuf'] = shuf_df["shuffled_column"].apply(extract_id)
        shuf_df['spk_id_shuf'] = shuf_df["shuffled_column"].apply(extract_spk_id)


        new_columns = ['ID', 'spk_id', 'wav','ID_shuf','spk_id_shuf',"shuffled_column"]
        shuf_df = shuf_df.reindex(columns=new_columns)

#         print(shuf_df.head())

        unmatched_df = unmatched_df.append(shuf_df)
    print("No. of unmatched pairs",len(unmatched_df))   
        
    #fn to set the labels

    def get_label(id1,id2):
        label=""
        if (id1 == id2):
            label = 1
        else:
            label = 0
        return(int(label))


    final_df = unmatched_df.append(matched_df, ignore_index=True)
    print(final_df.head(5))
    
#     final_df["label"] = final_df.apply(lambda x: get_label(x["ID"], x["shuffled_column"]), axis=1)
    final_df["label"] = final_df.apply(lambda x: get_label(x["spk_id"], x["spk_id_shuf"]), axis=1)

    final_df=final_df.drop(columns=["spk_id","ID","spk_id_shuf","ID_shuf"])

    veri_test = final_df.reindex(columns=['label', 'wav', 'shuffled_column'])

    #save the data frame as csv 
    print(veri_test.head())
    print("length of veri test",len(veri_test))
    print("Saving veri_test")
    veri_test.to_csv(output_folder+"veri_test.txt", sep=" ", index=False, header=False)
    print("Saved veri_test")
    
    

csv prep

In [6]:
def csv_prep (data_folder, df: pd.DataFrame):
    
    df["start"]=" "
    
    groups = df.groupby(df['ID'].apply(lambda x: x.split('_')[0]))
    
    output = []
    for spk, group in groups:
        for index, row in group.iterrows():
            id = row['ID'].split('_')[1]
#             wav = data_folder +spk+ "/" + id +  ".wav"
            wav = data_folder + "/" + id +  ".wav"
            #print(wav)

    
            signal, fs = torchaudio.load(wav)
            signal = signal.squeeze(0)
            audio_duration = signal.shape[0] / 16000
            start_sample = 0
            stop_sample = signal.shape[0]
            stop_sample= int(stop_sample)
            #print(stop_sample)

            df.loc[index, 'start'] = start_sample
            df.loc[index, "stop"] = stop_sample
            df.loc[index, "duration"] = audio_duration 
 
            
    df['stop'] = df['stop'].astype(int)
    
    df_f = df.reindex(columns=['ID', 'duration', 'wav', "start","stop","spk_id"])

    #print("num_frames")
    #num_frames=df['stop']-df['start']
    #print(num_frames)
    
    return df_f

In [7]:
def main (split, utt2spk, wavscp, audio_dir, output_folder):
#Make the main dataframe

    #utt2spk prep
    d1=pd.read_csv(utt2spk, header=None)
    d1.columns= ["col1"]
    d1[['ID', 'spk_id']] = d1['col1'].str.split(' ', expand=True)
    d1 = d1.drop(columns=["col1"])

    #wav.scp prep
    d2=pd.read_csv(wavscp, header=None)
    d2.columns= ["col1"]
    d2[['ID', 'wav']] = d2['col1'].str.split(' ', expand=True)
    d2 = d2.drop(columns=["col1"])

    #main df 
    d= pd.merge(d1, d2, on="ID")
    #print(d.head())
    #d['wav'] = d['wav'].apply(lambda x: str(x).replace( '/home/sai/work/silver-bullet/batch-02/','/path/wav/'))
    print("Length of d",len(d))
    #mask = d["wav"].isin(["/mnt/datadrive/datasets/spkveri/mrt1/spk450/wav/16783193/281474985386592.wav",..]
    ignore_ids=["281474985386592.wav","281474985386582.wav", 
                "281474985386570.wav","281474985386581.wav",
                "281474985436102.wav","281474989055448.wav"]
    mask = d["wav"].isin([audio_dir+"/"+ids for ids in ignore_ids])
    d = d[~mask]
    print("Length of new d",len(d))

    print("Made main data frame")


#split fn 
    print("Split into train dev test enrol")
    train_df, validation_df, test_df, enrol_df = split_fn(d, split)
    print("Done")

#     test_df.head(2)
    
    
#Prep veri_test
    print("Start making veri_test")
    veri_test(test_df,output_folder)
    print("Saved veri_test")


#Prep 4 csv files
    print("start making the csv files")
    print("Saving train.csv")
    traindf=csv_prep(data_folder,train_df)
    traindf.to_csv(output_folder+"train.csv", index= False)
    print("Saved train.csv")

    print("Saving dev.csv")
    devdf=csv_prep(data_folder,validation_df)
    devdf.to_csv(output_folder+"dev.csv",index= False)
    print("Saved dev.csv")

    print("Saving test.csv")
    testdf=csv_prep(data_folder,test_df)
    testdf.to_csv(output_folder+"test.csv",index= False)
    print("Saved test.csv")

    print("Saving enrol.csv")
    enroldf=csv_prep(data_folder,enrol_df)

#     print(enroldf.loc[enroldf['wav'] == 
#         '/mnt/datadrive/datasets/spkveri/mrt1/spk450/wav/16783193/281474985386592.wav'])

    enroldf.to_csv(output_folder+"enrol.csv",index= False)
    print("Saved enrol.csv")

    print("-----------------Done preprocessing!--------------------") 
#     return(test_df)

In [8]:
# main(split,utt2spk,wavscp,data_folder,output_folder)
main(split,utt2spk,wavscp,audio_dir,output_folder)

Length of d 250
Length of new d 250
Made main data frame
Split into train dev test enrol
[150, 50, 50]
next index: 200
Length of train_df 150
Length of dev 50
Length of test 50
Done
Start making veri_test
no of matched pairs 54
                                                 wav  \
0  /mnt/results/silverbullet/audios_500/281474988...   
1  /mnt/results/silverbullet/audios_500/281474989...   

                                     shuffled_column  
0  /mnt/results/silverbullet/audios_500/281474988...  
1  /mnt/results/silverbullet/audios_500/281474989...  
                         ID  spk_id                   ID_shuf  spk_id_shuf  \
0  16783229_281474988280618     NaN  16783229_281474988280615          NaN   
1  16794032_281474989340217     NaN  16794032_281474989340214          NaN   
2  16783229_281474988280618     NaN  16783229_281474988280626          NaN   
3  16794032_281474989340225     NaN  16794032_281474989340211          NaN   
4  16783229_281474988280615     NaN  16783229_28

/tmp/ipykernel_3037233/4290807906.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matched_df = matched_df.append(shufmatched_df, ignore_index=True)
/tmp/ipykernel_3037233/4290807906.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matched_df = matched_df.append(shufmatched_df, ignore_index=True)
/tmp/ipykernel_3037233/4290807906.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matched_df = matched_df.append(shufmatched_df, ignore_index=True)
/tmp/ipykernel_3037233/4290807906.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matched_df = matched_df.append(shufmatched_df, ignore_index=True)
/tmp/ipykernel_3037233/429080790

/tmp/ipykernel_3037233/4290807906.py:97: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unmatched_df = unmatched_df.append(shuf_df)
/tmp/ipykernel_3037233/4290807906.py:97: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unmatched_df = unmatched_df.append(shuf_df)
/tmp/ipykernel_3037233/4290807906.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = unmatched_df.append(matched_df, ignore_index=True)
/tmp/ipykernel_3037233/283853947.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

No. of unmatched pairs 11040
                         ID    spk_id  \
0  16794032_281474989340213  16794032   
1  16794032_281474989340213  16794032   
2  16783229_281474988280637  16783229   
3  16783229_281474988280637  16783229   
4  16794032_281474989340213  16794032   

                                                 wav  \
0  /mnt/results/silverbullet/audios_500/281474989...   
1  /mnt/results/silverbullet/audios_500/281474989...   
2  /mnt/results/silverbullet/audios_500/281474988...   
3  /mnt/results/silverbullet/audios_500/281474988...   
4  /mnt/results/silverbullet/audios_500/281474989...   

                    ID_shuf spk_id_shuf  \
0  16783229_281474988280637    16783229   
1  16783229_281474988280637    16783229   
2  16794032_281474989340213    16794032   
3  16794032_281474989340213    16794032   
4  16783229_281474988280637    16783229   

                                     shuffled_column  
0  /mnt/results/silverbullet/audios_500/281474988...  
1  /mnt/results/si

/tmp/ipykernel_3037233/283853947.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stop'] = df['stop'].astype(int)
/tmp/ipykernel_3037233/283853947.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["start"]=" "
/tmp/ipykernel_3037233/283853947.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 's